In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd drive

/content/drive


In [0]:
cd My\ Drive

/content/drive/My Drive


In [0]:
cd OurCode/

/content/drive/My Drive/OurCode


In [0]:
import re
import pandas as pd
import numpy as np
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.models import Sequential

from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D

from keras.callbacks import Callback
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
#from keras.utils.vis_utils import plot_model

from sklearn.utils import shuffle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [0]:
data = pd.read_csv("BigTraining.csv",header=None)
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [0]:
del data[1],data[2],data[3],data[4]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [0]:
data[0] = data[0].apply(lambda x: 1 if x == 4 else 0)

In [0]:
data.columns = ["sentiment","tweet"]
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,sentiment,tweet
0,1,@Kekeluvsday26 Thanks
1,0,"@felixmeister yeah, wish i could pull that one"
2,1,@epidaetia whats bad abt it? good thing indeed...
3,0,My tummy hurts
4,1,~@OzAtheist It was a keystroke error... honest


In [0]:
data.head()

,sentiment,tweet
0,1,@Kekeluvsday26 Thanks
1,0,"@felixmeister yeah, wish i could pull that one"
2,1,@epidaetia whats bad abt it? good thing indeed...
3,0,My tummy hurts
4,1,~@OzAtheist It was a keystroke error... honest


In [0]:
#####################################################
################# Utility Functions #################
#####################################################

def replaceUserName(tweet):
	tweet = re.sub('@[^\s]+','USERNAME',tweet)
	return tweet

def replaceLink(tweet):
	tweet = re.sub('((www\.[^\s]+)|((https|http)?://[^\s]+))','URL',tweet)
	return tweet

def replaceWithTwoChar(tweet):
    pattern = re.compile(r"(.)\1{1,}") 
    tweet = pattern.sub(r"\1\1", tweet)
    return tweet
  
stopWords = list(set(stopwords.words("english")))
stopWords = [str(word) for word in stopWords]
MAXLEN = float('-inf')

def cleanTweet(tweet):
  global MAXLEN
  tweet = tweet.strip().lower()
  tweet = replaceUserName(tweet)
  tweet = replaceLink(tweet)
  tweet = replaceWithTwoChar(tweet)
  wordsInTweet = tweet.split()
  #Removing StopWords to reduce no. of features
  #text = ' '.join([word for word in wordsInTweet if word not in stopWords])
  #wordsInTweet =  text.split()

  if MAXLEN < len(wordsInTweet):
    MAXLEN = len(wordsInTweet)
  return wordsInTweet

In [0]:
cleanTweet(data['tweet'][0])

['USERNAME', 'thanks']

In [0]:
data['tokens'] = data['tweet'].apply(lambda x: cleanTweet(x))
data.head()

,sentiment,tweet,tokens
0,1,@Kekeluvsday26 Thanks,"[USERNAME, thanks]"
1,0,"@felixmeister yeah, wish i could pull that one","[USERNAME, yeah,, wish, i, could, pull, that, ..."
2,1,@epidaetia whats bad abt it? good thing indeed...,"[USERNAME, whats, bad, abt, it?, good, thing, ..."
3,0,My tummy hurts,"[my, tummy, hurts]"
4,1,~@OzAtheist It was a keystroke error... honest,"[~USERNAME, it, was, a, keystroke, error.., ho..."


In [0]:
del data['tweet']

In [0]:
MAX_NB_WORDS = 90000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
total_size = 800000
test_total_size = 150000
tokenizer.fit_on_texts(data['tokens'][:total_size])

In [0]:

batch_size = total_size/100
number_of_batches = 100
train_seq = list()
for i in range(number_of_batches):
  print i+1,"batch"
  batch_seq = tokenizer.texts_to_sequences(data["tokens"][i*batch_size:(i+1)*batch_size])
  train_seq = np.concatenate((train_seq,batch_seq))
del batch_seq

1 batch
2 batch
3 batch
4 batch
5 batch
6 batch
7 batch
8 batch
9 batch
10 batch
11 batch
12 batch
13 batch
14 batch
15 batch
16 batch
17 batch
18 batch
19 batch
20 batch
21 batch
22 batch
23 batch
24 batch
25 batch
26 batch
27 batch
28 batch
29 batch
30 batch
31 batch
32 batch
33 batch
34 batch
35 batch
36 batch
37 batch
38 batch
39 batch
40 batch
41 batch
42 batch
43 batch
44 batch
45 batch
46 batch
47 batch
48 batch
49 batch
50 batch
51 batch
52 batch
53 batch
54 batch
55 batch
56 batch
57 batch
58 batch
59 batch
60 batch
61 batch
62 batch
63 batch
64 batch
65 batch
66 batch
67 batch
68 batch
69 batch
70 batch
71 batch
72 batch
73 batch
74 batch
75 batch
76 batch
77 batch
78 batch
79 batch
80 batch
81 batch
82 batch
83 batch
84 batch
85 batch
86 batch
87 batch
88 batch
89 batch
90 batch
91 batch
92 batch
93 batch
94 batch
95 batch
96 batch
97 batch
98 batch
99 batch
100 batch


In [0]:
print len(train_seq)
print MAXLEN

800000
64


In [0]:
MAXLEN += 10
padded_train_sequences = pad_sequences(train_seq, maxlen=MAXLEN)
del train_seq

In [0]:
len(padded_train_sequences)

800000

In [0]:
y_train = data['sentiment'][:total_size]

In [0]:
len(y_train)

800000

In [0]:
y_test = data['sentiment'][total_size:total_size+test_total_size]
len(y_test)

150000

In [0]:
def get_rnn_cnn_model():
    embedding_dim = 300
    inp = Input(shape=(MAXLEN, ))
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAXLEN, trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

rnn_cnn_model = get_rnn_cnn_model()

#plot_model(rnn_cnn_model, to_file='./images/article_5/rnn_cnn_model.png', show_shapes=True, show_layer_names=True)

#plot_model(cnn_model_multi_channel, to_file='./images/article_5/cnn_model_multi_channel.png', show_shapes=True, show_layer_names=True)


In [0]:
batch_size = test_total_size/50
number_of_batches = 50
test_seq = list()
for i in range(number_of_batches):
  print i+1,"batch"
  batch_seq = tokenizer.texts_to_sequences(data["tokens"][total_size+i*batch_size:total_size+(i+1)*batch_size])
  test_seq = np.concatenate((test_seq,batch_seq))
del batch_seq
print len(test_seq)

1 batch
2 batch
3 batch
4 batch
5 batch
6 batch
7 batch
8 batch
9 batch
10 batch
11 batch
12 batch
13 batch
14 batch
15 batch
16 batch
17 batch
18 batch
19 batch
20 batch
21 batch
22 batch
23 batch
24 batch
25 batch
26 batch
27 batch
28 batch
29 batch
30 batch
31 batch
32 batch
33 batch
34 batch
35 batch
36 batch
37 batch
38 batch
39 batch
40 batch
41 batch
42 batch
43 batch
44 batch
45 batch
46 batch
47 batch
48 batch
49 batch
50 batch
150000


In [0]:
padded_test_sequences = pad_sequences(test_seq, maxlen=MAXLEN)
del test_seq
print len(padded_test_sequences),len(padded_test_sequences[0])

150000 74


In [0]:
batch_size = 256
epochs = 4
history = rnn_cnn_model.fit(x=padded_train_sequences, 
                    y=y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1)

Epoch 1/4
800000/800000 [==============================] - 1092s 1ms/step - loss: 0.4603 - acc: 0.7787
Epoch 2/4
800000/800000 [==============================] - 1089s 1ms/step - loss: 0.3879 - acc: 0.8256
Epoch 3/4
800000/800000 [==============================] - 1089s 1ms/step - loss: 0.3561 - acc: 0.8425
Epoch 4/4
800000/800000 [==============================] - 1092s 1ms/step - loss: 0.3295 - acc: 0.8573


In [0]:
y_pred_rnn = rnn_cnn_model.predict(padded_test_sequences, verbose=1, batch_size=2048)

y_pred_rnn = pd.DataFrame(y_pred_rnn, columns=['prediction'])
print y_pred_rnn.head()
y_pred_rnn['prediction'] = y_pred_rnn['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
#y_pred_cnn_multi_channel.to_csv('./predictions/y_pred_cnn_multi_channel.csv', index=False)

150000/150000 [==============================] - 14s 96us/step
   prediction
0    0.930815
1    0.622040
2    0.994880
3    0.300449
4    0.037942


In [0]:
from sklearn.metrics import accuracy_score
#y_pred_cnn_multi_channel = pd.read_csv('./predictions/y_pred_cnn_multi_channel.csv')
print(accuracy_score(y_test, y_pred_rnn))

0.8208266666666667


In [0]:
rnn_cnn_model.save("cnn_rnn_stopwords_notRemoved.h5")